In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date, time
import urllib
import json
import ssl

---

### **Import the data**

Import CSV files

In [2]:
df_bonn = pd.read_csv("data/bonn.csv")
df_essen = pd.read_csv("data/essen.csv")

In [3]:
df_bonn.head()

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng
0,2019-03-06,01:13:00,21169,bonn,0 days 02:51:00.000000000,50.921682,6.959204,50.953793,6.899248
1,2019-03-06,14:43:00,21169,bonn,0 days 00:11:00.000000000,50.953793,6.899251,50.947463,6.922385
2,2019-03-06,15:21:00,21169,bonn,0 days 00:21:00.000000000,50.947462,6.922391,50.929703,6.932212
3,2019-03-06,17:45:00,21169,bonn,0 days 00:51:00.000000000,50.929703,6.932212,50.945799,6.915687
4,2019-03-07,05:58:00,21169,bonn,0 days 00:07:00.000000000,50.945765,6.915829,50.951143,6.915550


In [4]:
df_bonn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329629 entries, 0 to 329628
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   day            329629 non-null  object 
 1   time           329629 non-null  object 
 2   b_number       329629 non-null  int64  
 3   city           329629 non-null  object 
 4   trip_duration  329629 non-null  object 
 5   orig_lat       329629 non-null  float64
 6   orig_lng       329629 non-null  float64
 7   dest_lat       329629 non-null  float64
 8   dest_lng       329629 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 22.6+ MB


In [5]:
# Check df_essen for any trip_durations longer than a day
for i in list(df_essen["trip_duration"]):
    if i[0] != "0":
        print(i[0])

Import API data

In [ ]:
context = ssl._create_unverified_context()

def request(city_id):
    response = urllib.request.urlopen(f"https://api.nextbike.net/maps/nextbike-live.json?city={city_id}", context=context)
    response = response.read().decode("utf-8")
    return json.loads(response)

bonn_json = request(547)["countries"][0] # Bonn city ID at NextBike is 547
essen_json = request(158)["countries"][0] # Essen city ID at NextBike ist 133
# JSONs only contain one key on 0th level, "countries", which only contains a list with 1 element

In [ ]:
# Use as center points for maps:
bonn = np.array([bonn_json["lat"], bonn_json["lng"]])
essen = np.array([essen_json["lat"], essen_json["lng"]])

---

### **Format the data**

In [ ]:
def format_trip_duration(df):
    """
    Assumption: all time durations are less than 1 day
    Assumption: all time durations are measured in seconds (not milliseconds)
    First turn "0 days 02:51:00.0000" into list split by " ", so we get 
    x=["0", "days", "02:51:00.0000"].
    Since all trip_durations are less than 1 day, we can discard x[0] and x[1] and 
    focus on x[2].
    We then split x[2], i.e. "02:51:00.0000", into a list split by ":", so we get 
    x=["02", "51", "00.0000"].
    We now turn these elements into numbers and pass them into the timedelta constructor.
    Timedelta objects describe differences in time, like time durations.
    """
    df["trip_duration"] = df["trip_duration"].apply(
        lambda s: s.split(" ")[2].split(":"))
    df["trip_duration"] = df["trip_duration"].apply(
        lambda x: timedelta(hours=int(x[0]), minutes=int(x[1]), seconds=int(float(x[2]))))
    return df

def str_to_date(row):
    return datetime.strptime(f"{row['day']} {row['time']}", "%Y-%m-%d %H:%M:%S")

In [ ]:
%%time
# Sort values, reset index (and drop old index)
# Bonn:
df_bonn.sort_values(["day", "time"], inplace=True)
df_bonn = df_bonn.reset_index()
df_bonn = df_bonn.drop("index", axis=1)
# Essen:
df_essen.sort_values(["day", "time"], inplace=True)
df_essen = df_essen.reset_index()
df_essen = df_essen.drop("index", axis=1)

# Format trip duration
df_bonn = format_trip_duration(df_bonn)
df_essen = format_trip_duration(df_essen)

# Create timestamp, weekday, hour columns
# Bonn:
df_bonn["timestamp"] = df_bonn.apply(str_to_date, axis=1)
df_bonn["weekday"] = df_bonn["timestamp"].apply(lambda row: row.weekday())
df_bonn["hour"] = df_bonn["timestamp"].apply(lambda row: row.hour)
# Essen:
df_essen["timestamp"] = df_essen.apply(str_to_date, axis=1)
df_essen["weekday"] = df_essen["timestamp"].apply(lambda row: row.weekday())
df_essen["hour"] = df_essen["timestamp"].apply(lambda row: row.hour)

# Combine longitude/latitude coordinate columns: orig
# Bonn:
df_bonn["orig"] = list(zip(df_bonn["orig_lat"].round(4), df_bonn["orig_lng"].round(4)))
df_bonn["orig"] = df_bonn["orig"].apply(np.array) # turn tuples in np.arrays
df_bonn["dest"] = list(zip(df_bonn["dest_lat"].round(4), df_bonn["dest_lng"].round(4)))
df_bonn["dest"] = df_bonn["dest"].apply(np.array) # turn tuples in np.arrays
# Essen:
df_essen["orig"] = list(zip(df_essen["orig_lat"].round(4), df_essen["orig_lng"].round(4)))
df_essen["orig"] = df_essen["orig"].apply(np.array) # turn tuples in np.arrays
df_essen["dest"] = list(zip(df_essen["dest_lat"].round(4), df_essen["dest_lng"].round(4)))
df_essen["dest"] = df_essen["dest"].apply(np.array) # turn tuples in np.arrays

In [ ]:
df_bonn

In [ ]:
df_bonn.info()